In [ ]:
# 📦 Required: pip install fredapi scikit-learn pandas matplotlib

from fredapi import Fred
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os

# 🔧 Setup
API_KEY = "YOUR API KEY HERE"  # Replace with your own if needed
fred = Fred(api_key=API_KEY)
scaler = MinMaxScaler()

# 🗓️ Timeframe
start_date = "2000-01-01"
end_date = "2025-01-01"
freq = "MS"
date_index = pd.date_range(start=start_date, end=end_date, freq=freq)
df = pd.DataFrame(index=date_index)

# ✅ Feature codes and names
features = {
    "GASREGCOVW": "layman__gas_price",
    "MSPUS": "layman__housing_prices",
    "RSAFS": "layman__retail_sales",
    "MORTGAGE30US": "layman__mortgage_rate",
    "TERMCBAUTO48NS": "layman__auto_loan_rate",
    "CUSR0000SETA02": "layman__used_car_prices",
    "CUSR0000SEHF01": "layman__electricity_cost_index",
    "TOTALSL": "layman__consumer_credit",
    "SLOAS": "layman__student_debt_level",
    "RRVRUSQ156N": "layman__rental_vacancy_rate",  # Quarterly → needs interpolation
    "CUSR0000SAF1": "layman__grocery_index",
    "CUSR0000SETG01": "layman__airfare_index"
}

# 📥 Pull and normalize
for code, col_name in features.items():
    try:
        series = fred.get_series(code, observation_start=start_date, observation_end=end_date)
        monthly_series = series.resample(freq).ffill().reindex(date_index)
        scaled = scaler.fit_transform(monthly_series.values.reshape(-1, 1))
        df[col_name] = pd.Series(scaled.flatten(), index=date_index).clip(0.01, 0.99)
        print(f"✅ Loaded {col_name}")
    except Exception as e:
        print(f"❌ Failed to load {col_name}: {e}")
        df[col_name] = pd.NA

# 💾 Save
os.makedirs("layman_batches", exist_ok=True)
df.to_csv("layman_batches/Batch_Layman_FRED_Expanded.csv")
print("✅ Saved as 'Batch_Layman_FRED_Expanded.csv'")

In [ ]:
# 📦 Imports
from fredapi import Fred
import pandas as pd
import os

# 🔧 Setup
API_KEY = "YOUR API KEY HERE"
fred = Fred(api_key=API_KEY)
start_date = "2000-01-01"
end_date = "2025-01-01"
freq = "M"  # Monthly frequency

# ✅ Feature mapping: FRED series → custom column names
features = {
    "TOTALSL": "layman__consumer_credit",                      # Total consumer loans
    "TERMCBAUTO48NS": "layman__auto_loan_rate",                # Avg 48-month new auto loan rate
    "SLOAS": "layman__student_debt_level",                     # Total student loan debt
    "RRVRUSQ156N": "layman__rental_vacancy_rate",              # Quarterly → ffill to monthly
    "CUUR0000SETA03": "layman__fast_food_price_index",         # CPI: Limited Service Meals
    "CUSR0000SEEA": "layman__concert_ticket_index",            # CPI: Admission to events
    "CUSR0000SETG01": "layman__airfare_index",                 # CPI: Airline fares
    "CUSR0000SEEB03": "layman__childcare_cost_index",          # CPI: Childcare
    "CUSR0000SETG": "layman__public_transit_fares",            # CPI: Public transit
}

# 🗓️ Create monthly date index and base DataFrame
date_index = pd.date_range(start=start_date, end=end_date, freq=freq)
df = pd.DataFrame(index=date_index)

# 📥 Download FRED data
for code, alias in features.items():
    try:
        series = fred.get_series(code, observation_start=start_date, observation_end=end_date)
        series = series.resample(freq).ffill()
        df[alias] = series
        print(f"✅ Loaded {alias}")
    except Exception as e:
        print(f"❌ Failed to load {alias}: {e}")

# 🧠 Netflix Subscriptions Placeholder
df["layman__netflix_subs_index"] = pd.NA
print("🧱 Placeholder added: layman__netflix_subs_index")

# 🧼 Final formatting
df.index.name = "date"
df = df.ffill()

# 💾 Save to CSV
os.makedirs("linchpin_batches", exist_ok=True)
df.to_csv("../linchpin_batches/Batch_Layman_02.csv")
print("✅ Saved Batch_Layman_02.csv")